# Tensor Board

## 1 Graph
tf.Graph() # 그래프 추가

tf.get_default_graph() # 그래프 초기화

tf.name_scope() # Operation 을 block으로 묶는다 (시각화에 용이)

### 01 Intro

In [1]:
import tensorflow as tf

a = tf.add(2,5)
b = tf.multiply(a,3)

sess = tf.Session(graph=tf.get_default_graph())  #sess = tf.Session() 과 동일
sess.run(tf.global_variables_initializer()) # 텐서플로 객체 값들을 초기화 (가비징))
print(sess.run(b))
print(sess.run([a,b]))   # 여러 연산은 list 객체로 입력
sess.close()

writer = tf.summary.FileWriter('./my_graph', sess.graph) #'./my_graph' 객체 저장 폴더명
writer.close()
# ! tensorboard --logdir='my_graph'  # 실행파일 폴더로 이동 후 실행

21
[7, 21]


### 02 Name_Scope
Name_Scope

In [2]:
# Two name_scope
import tensorflow as tf
with tf.name_scope('Scope_A'):
    a = tf.add(1,2, name='A_add')
    b = tf.add(a,3, name='A_mul')
    
with tf.name_scope("Scope_B"):
    c = tf.add(4,5, name="B_add")
    d = tf.add(c,6, name="B_mul")

e = tf.add(b,d, name="output")

# 그래프를 즉시 Export 하므로 , Tensorboard를 바로 시작이 가능하다
writer = tf.summary.FileWriter('./name_scope_1', graph=tf.get_default_graph())
writer.close()

### 03 Name_Scope
Name_Scope in Name_Scope

In [3]:
graph = tf.Graph()

with graph.as_default():
    in_1 = tf.placeholder(tf.float32, shape=[], name = 'input_a')
    in_2 = tf.placeholder(tf.float32, shape=[], name = 'input_b')
    const = tf.constant(3, dtype=tf.float32, name="static_value")
    
    with tf.name_scope("A"):
        A_mul = tf.multiply(in_1, const); A_out = tf.subtract(A_mul, in_1)
        
    with tf.name_scope("B"):
        B_mul = tf.multiply(in_2, const); B_out = tf.subtract(B_mul, in_2)
    
    with tf.name_scope("C"):
        C_div = tf.div(A_out, B_out); C_out = tf.add(C_div, const)

    with tf.name_scope("D"):
        D_div = tf.div(B_out, A_out); D_out = tf.add(D_div, const)

    # Operation 최종 출력노드
    out = tf.maximum(C_out, D_out)

writer = tf.summary.FileWriter('./name_scope_2', graph=graph)
writer.close()

### 04 Excercise

In [4]:
import tensorflow as tf
graph = tf.Graph() # Explicitly create a Graph object

In [5]:
with graph.as_default():
    with tf.name_scope("variables"):
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="total_output")

        with tf.name_scope("transformation"): # Primary transformation Operations
            with tf.name_scope("input"):      # Separate input layer
                a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
            with tf.name_scope("intermediate_layer"):   # Separate middle layer
                b = tf.reduce_prod(a, name="product_b")
                c = tf.reduce_sum(a, name="sum_c")
                with tf.name_scope("output"):           # Separate output layer
                    output = tf.add(b, c, name="output")
                with tf.name_scope("update"):
                    update_total = total_output.assign_add(output)
                    increment_step = global_step.assign_add(1)
                    
    with tf.name_scope("summaries"):     # Summary Operations
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        #tf.summary
        tf.summary.scalar("output_summary", output) # Creates summaries for output node
        tf.summary.scalar("total_summary", update_total)
        tf.summary.scalar("average_summary", avg)
        
    with tf.name_scope("global_ops"):               # Global Variables and Operations
        init = tf.global_variables_initializer()    # Initialization Op
        merged_summaries = tf.contrib.deprecated.merge_all_summaries()
        #tf.merge_all_summaries() # Merge all summaries into one Operation

Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.merge.


In [6]:
sess = tf.Session(graph=graph)  # Start a Session
writer = tf.summary.FileWriter('./improved_graph', graph)

In [7]:
sess.run(init)
def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, merged_summaries],
                                feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [8]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])
# Write the summaries to disk
writer.flush()
# Close the SummaryWriter
writer.close()
# Close the session
sess.close()